In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
import random

In [22]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, RNN, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import pandas as pd
np.random.seed(1)

### Helper functions for Data Loading, normalization, etc

In [3]:
def Reshape_to_input(X, T, Mat_dim):
    """
    reshapes data
    """
   
    [Num_examples, dim] = X.shape
    input_X = X[:,2:].reshape(Num_examples, Mat_dim, T)
    
    return input_X
    

In [4]:
def NormalizeTowardsMean(X):
   # print(X.shape)
    "# Preprocessing: Subtract the mean feature - a question if it is feasible - all features are already in the same scale...\n",
    # TODO: try -1 1 normalization
    mean_feat = np.mean(X, axis=0, keepdims=True)
   # print(mean_feat.shape)
    X -= mean_feat
    # Preprocessing: Divide by standard deviation. This ensures that each feature\n",
    # has roughly the same scale.\n",
    std_feat = np.std(X, axis=0, keepdims=True)
    X /= std_feat
    
    return X



In [5]:
def Random_Selection_train_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    Persons_train = np.array([])
    Persons_test = np.array([])
    #print(X_train.shape)
    for Person in range(0,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, returned_train
            
            


In [6]:
def Random_Selection_train_val_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    X_val = np.array([])
    Y_val = np.array([])
    Persons_train = np.array([])
    Persons_val = np.array([])
    Persons_test = np.array([])
    
    train_N =int(np.ceil(N*2/3))
    
    for Person in range(0,train_N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
        Persons_train = np.append(Persons_train, label, axis = 0)
    X_train=X_train.reshape(Y_train.size, num_dim)    
    
     
    for Person in range(train_N,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_val= np.append(X_val,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_val = np.append(Y_val, X_all[[indexes], 0])
        Persons_val = np.append(Persons_val, label, axis = 0)
    X_val=X_val.reshape(Y_val.size, num_dim)  
    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
        Persons_test = np.append(Persons_test, label, axis = 0)
    X_test=X_test.reshape(Y_test.size, num_dim) 
    print('Persons for train: ')
    print(np.unique(Persons_train))
    returned_train = np.unique(Persons_train)
    print('Persons for val: ')
    print(np.unique(Persons_val))
    returned_val = np.unique(Persons_val)
    print('Persons for test: ')
    print(np.unique(Persons_test))
    return X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val
            
            


In [7]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, int(inputY[i]-1)] = 1
        
    
    return Y


In [8]:
def Gait_model(data):
    """
    Function creating the Gait model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    units = 128 
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.

    X = Bidirectional(LSTM(units, return_sequences=True))( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
   
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(units, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [9]:
# load my dataset and parameters from a file
from data_utils import loadfromfolder

### Model Training

In [23]:
myoptim=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filepath, save_best_only =True, monitor = 'val_loss', mode ='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min')
    return [es, mcp_save, reduce_lr_loss]

In [19]:
myoptim=SGD(lr=0.01, decay=1e-8, momentum=0.9, nesterov=True)

def get_callbacks(filepath, patience=2):
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
    mcp_save = ModelCheckpoint(filepath, save_best_only =True, monitor = 'val_loss', mode ='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose=1, epsilon=1e-4, mode='min')
    return [es, mcp_save, reduce_lr_loss]

NameError: name 'SGD' is not defined

In [25]:
# parameters needed
trials = 5
epochs = 50
batch_size = 22
units = 128 # actually defind inside the funciton of the model - TODO - make it here
path ='data/best'
Files, T, max_dim = loadfromfolder()
#Where Files are names, T - my division into frames and max_dim - feature dimensionality
fRes = open("Results.txt","a+") 
fRes.write("Number of units in LSTM = %d,  Epochs =  %d and batch size =  %d \r\n" % (units, epochs,batch_size))
for file in range(0, len(Files)):
    name = Files[file]
    t = T[file]
    dim = max_dim[file]
    name_w_path = path +"/"+name
    fRes.write("Experiment %d from file %s \r\n" % (file,name))
    # here all the routine in One cell 
    X_all = pd.read_csv(name_w_path)
    # the routine to run the same test N times, randomly shuffling the data
    Accuracy  = np.zeros(shape=(trials))
        
    for i in range(0,trials):
        modelname = str(file)+str(i)
        X_train, Y_train, X_test, Y_test, X_val, Y_val, returned_train, returned_val = Random_Selection_train_val_test (X_all.values, 16)
        #namePersons = str(i)+'_'+'file'+ name[:-4] +'.txt'
        #np.savetxt(namePersons, returned_train, delimiter=',', fmt='%d')   # X is an array
        X_train = NormalizeTowardsMean(X_train) # substract mean from train set all the time
        X_test = NormalizeTowardsMean(X_test)
        X_val = NormalizeTowardsMean(X_val)
        
        Y_train_oh = convert_to_one_hot(Y_train, C = 3)
        Y_test_oh = convert_to_one_hot(Y_test, C = 3)
        Y_val_oh = convert_to_one_hot(Y_val, C = 3)
        
        X_train = Reshape_to_input(X_train, t, dim)
        X_test = Reshape_to_input(X_test, t, dim)
        X_val = Reshape_to_input(X_val, t, dim)
        
        model = Gait_model((X_train[0,:,:]))
        
        file_path = 'models/file_{}_model_wts.hdf5'.format(modelname)
        callbacks = get_callbacks(filepath=file_path, patience = 35)
        model.compile(loss='categorical_crossentropy', optimizer=myoptim, metrics=['accuracy'])
              
        
        model.fit(X_train, Y_train_oh, epochs=epochs,\
                  batch_size=batch_size, shuffle=True, validation_data=(X_val, Y_val_oh), callbacks=callbacks)
        
        loss, acc = model.evaluate(X_test, Y_test_oh)
        
        Accuracy[i]=acc
        fRes.write("Final accuracy per trial %f \r\n" % (acc))
        
    print(Accuracy)
    fRes.write("Final mean accuracy is: %f \r\n" % (np.mean(Accuracy)))
fRes.close() 


angles_cycles_frames_15t_08f_angles_22persons.csv
covmat_cycles_frames_12t_36f_angles_22persons.csv
Persons for train: 
[ 1.  3.  7.  9. 13. 14. 15. 16. 19. 20. 22.]
Persons for val: 
[ 2.  4. 11. 18. 21.]
Persons for test: 
[ 5.  6.  8. 10. 12. 17.]


/home/khokhlov/virtenv/ml36/lib/python3.5/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 263 samples, validate on 136 samples
Epoch 1/50
263/263 [==============================] - 43s 164ms/step - loss: 1.0899 - acc: 0.4297 - val_loss: 1.0874 - val_acc: 0.5221
Epoch 2/50
263/263 [==============================] - 1s 5ms/step - loss: 1.0856 - acc: 0.4487 - val_loss: 1.0835 - val_acc: 0.5221
Epoch 3/50
263/263 [==============================] - 1s 5ms/step - loss: 1.0807 - acc: 0.4297 - val_loss: 1.0800 - val_acc: 0.5588
Epoch 4/50
263/263 [==============================] - 1s 5ms/step - loss: 1.0746 - acc: 0.4829 - val_loss: 1.0771 - val_acc: 0.5735
Epoch 5/50
263/263 [==============================] - 1s 5ms/step - loss: 1.0724 - acc: 0.4829 - val_loss: 1.0746 - val_acc: 0.5882
Epoch 6/50
263/263 [==============================] - 1s 4ms/step - loss: 1.0639 - acc: 0.5323 - val_loss: 1.0722 - val_acc: 0.5882
Epoch 7/50
263/263 [==============================] - 1s 5ms/step - loss: 1.0653 - acc: 0.5323 - val_loss: 1.0699 - val_acc: 0.5956
Epoch 8/50
263/263 [=======

274/274 [==============================] - 1s 5ms/step - loss: 1.0662 - acc: 0.4526 - val_loss: 1.0725 - val_acc: 0.5169
Epoch 11/50
274/274 [==============================] - 1s 5ms/step - loss: 1.0661 - acc: 0.4964 - val_loss: 1.0701 - val_acc: 0.5254
Epoch 12/50
274/274 [==============================] - 1s 4ms/step - loss: 1.0614 - acc: 0.5000 - val_loss: 1.0678 - val_acc: 0.5593
Epoch 13/50
274/274 [==============================] - 1s 4ms/step - loss: 1.0574 - acc: 0.5328 - val_loss: 1.0656 - val_acc: 0.5508
Epoch 14/50
274/274 [==============================] - 1s 4ms/step - loss: 1.0575 - acc: 0.5401 - val_loss: 1.0636 - val_acc: 0.5593
Epoch 15/50
274/274 [==============================] - 1s 4ms/step - loss: 1.0503 - acc: 0.5657 - val_loss: 1.0616 - val_acc: 0.5678
Epoch 16/50
274/274 [==============================] - 1s 4ms/step - loss: 1.0446 - acc: 0.5912 - val_loss: 1.0596 - val_acc: 0.5932
Epoch 17/50
274/274 [==============================] - 1s 4ms/step - loss: 1.0442

Epoch 20/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0273 - acc: 0.5729 - val_loss: 1.0412 - val_acc: 0.5610
Epoch 21/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0230 - acc: 0.6102 - val_loss: 1.0397 - val_acc: 0.5610
Epoch 22/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0182 - acc: 0.6136 - val_loss: 1.0381 - val_acc: 0.5610
Epoch 23/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0169 - acc: 0.6237 - val_loss: 1.0366 - val_acc: 0.5610
Epoch 24/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0162 - acc: 0.6407 - val_loss: 1.0351 - val_acc: 0.5610
Epoch 25/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0127 - acc: 0.5966 - val_loss: 1.0337 - val_acc: 0.5691
Epoch 26/50
295/295 [==============================] - 1s 4ms/step - loss: 1.0129 - acc: 0.6271 - val_loss: 1.0323 - val_acc: 0.5772
Epoch 27/50
295/295 [==============================] - 1s 4ms/step - 

...

Epoch 40/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0136 - acc: 0.5937 - val_loss: 1.0073 - val_acc: 0.6617
Epoch 41/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0079 - acc: 0.5833 - val_loss: 1.0062 - val_acc: 0.6617
Epoch 42/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0155 - acc: 0.5486 - val_loss: 1.0051 - val_acc: 0.6617
Epoch 43/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0073 - acc: 0.6181 - val_loss: 1.0041 - val_acc: 0.6617
Epoch 44/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0123 - acc: 0.5660 - val_loss: 1.0030 - val_acc: 0.6617
Epoch 45/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0073 - acc: 0.6007 - val_loss: 1.0020 - val_acc: 0.6617
Epoch 46/50
288/288 [==============================] - 1s 4ms/step - loss: 1.0066 - acc: 0.5937 - val_loss: 1.0009 - val_acc: 0.6541
Epoch 47/50
288/288 [==============================] - 1s 4ms/step - 

Epoch 49/50
291/291 [==============================] - 4s 13ms/step - loss: 1.0164 - acc: 0.6667 - val_loss: 1.0347 - val_acc: 0.6083
Epoch 50/50
136/136 [==============================] - 0s 3ms/step
Persons for train: 
[ 1.  3.  4.  6.  7.  9. 10. 11. 14. 17. 21.]
Persons for val: 
[ 2.  5. 12. 15. 18.]
Persons for test: 
[ 8. 13. 16. 19. 20. 22.]
Train on 291 samples, validate on 120 samples
Epoch 1/50
291/291 [==============================] - 48s 164ms/step - loss: 1.1155 - acc: 0.2680 - val_loss: 1.1002 - val_acc: 0.3750
Epoch 2/50
291/291 [==============================] - 4s 12ms/step - loss: 1.1078 - acc: 0.3058 - val_loss: 1.0979 - val_acc: 0.3750
Epoch 3/50
291/291 [==============================] - 4s 13ms/step - loss: 1.1040 - acc: 0.3505 - val_loss: 1.0960 - val_acc: 0.3917
Epoch 4/50
291/291 [==============================] - 4s 13ms/step - loss: 1.0988 - acc: 0.3436 - val_loss: 1.0944 - val_acc: 0.4000
Epoch 5/50
291/291 [==============================] - 4s 14ms/step -

271/271 [==============================] - 3s 12ms/step - loss: 1.0725 - acc: 0.5018 - val_loss: 1.0809 - val_acc: 0.5484
Epoch 7/50
271/271 [==============================] - 3s 12ms/step - loss: 1.0754 - acc: 0.4391 - val_loss: 1.0788 - val_acc: 0.5645
Epoch 8/50
271/271 [==============================] - 3s 11ms/step - loss: 1.0674 - acc: 0.4871 - val_loss: 1.0768 - val_acc: 0.5726
Epoch 9/50
271/271 [==============================] - 3s 12ms/step - loss: 1.0621 - acc: 0.5461 - val_loss: 1.0747 - val_acc: 0.5806
Epoch 10/50
271/271 [==============================] - 3s 12ms/step - loss: 1.0665 - acc: 0.4908 - val_loss: 1.0729 - val_acc: 0.5887
Epoch 11/50
271/271 [==============================] - 3s 12ms/step - loss: 1.0584 - acc: 0.5646 - val_loss: 1.0710 - val_acc: 0.6048
Epoch 12/50
271/271 [==============================] - 3s 12ms/step - loss: 1.0572 - acc: 0.5756 - val_loss: 1.0694 - val_acc: 0.6371
Epoch 13/50
271/271 [==============================] - 3s 12ms/step - loss: 1

279/279 [==============================] - 3s 12ms/step - loss: 1.0736 - acc: 0.4982 - val_loss: 1.0709 - val_acc: 0.5583
Epoch 16/50
279/279 [==============================] - 3s 12ms/step - loss: 1.0676 - acc: 0.5556 - val_loss: 1.0697 - val_acc: 0.5583
Epoch 17/50
279/279 [==============================] - 3s 12ms/step - loss: 1.0675 - acc: 0.5197 - val_loss: 1.0685 - val_acc: 0.5583
Epoch 18/50
279/279 [==============================] - 3s 12ms/step - loss: 1.0649 - acc: 0.5305 - val_loss: 1.0672 - val_acc: 0.5667
Epoch 19/50
279/279 [==============================] - 3s 12ms/step - loss: 1.0700 - acc: 0.5376 - val_loss: 1.0661 - val_acc: 0.5750
Epoch 20/50
279/279 [==============================] - 3s 12ms/step - loss: 1.0661 - acc: 0.5448 - val_loss: 1.0649 - val_acc: 0.5750
Epoch 21/50
279/279 [==============================] - 3s 12ms/step - loss: 1.0672 - acc: 0.5197 - val_loss: 1.0637 - val_acc: 0.5833
Epoch 22/50
279/279 [==============================] - 3s 11ms/step - loss

261/261 [==============================] - 3s 12ms/step - loss: 1.0462 - acc: 0.6245 - val_loss: 1.0565 - val_acc: 0.5691
Epoch 25/50
261/261 [==============================] - 3s 12ms/step - loss: 1.0369 - acc: 0.6475 - val_loss: 1.0555 - val_acc: 0.5772
Epoch 26/50
261/261 [==============================] - 3s 12ms/step - loss: 1.0390 - acc: 0.6705 - val_loss: 1.0546 - val_acc: 0.5772
Epoch 27/50
261/261 [==============================] - 3s 12ms/step - loss: 1.0411 - acc: 0.6437 - val_loss: 1.0538 - val_acc: 0.5772
Epoch 28/50
261/261 [==============================] - 3s 12ms/step - loss: 1.0428 - acc: 0.6207 - val_loss: 1.0529 - val_acc: 0.5772
Epoch 29/50
261/261 [==============================] - 3s 12ms/step - loss: 1.0375 - acc: 0.6513 - val_loss: 1.0520 - val_acc: 0.5772
Epoch 30/50
261/261 [==============================] - 3s 12ms/step - loss: 1.0366 - acc: 0.6552 - val_loss: 1.0511 - val_acc: 0.5772
Epoch 31/50
261/261 [==============================] - 3s 12ms/step - loss

In [12]:
fRes.close() 